In [ ]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import os

processed_file = 'data/processed_corpus.parquet'

if os.path.exists(processed_file):
    print(f"Loading processed data from {processed_file}...")
    df = pd.read_parquet(processed_file)


texts = df['processed_text'].tolist()

Loading processed data from data/processed_corpus.parquet...


In [ ]:
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

# Create the topic model
hdbscan_model = HDBSCAN(min_cluster_size=40, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
umap_model = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine', random_state=42)
vectorizer_model = CountVectorizer(
    tokenizer=lambda x: x.split(),
    preprocessor=None,
    token_pattern=None,
    ngram_range=(1, 2),
    min_df=5,  # Slightly higher to reduce noise
    max_df=0.95,  # Remove very common words
    max_features=5000  # Limit vocabulary size
)


topic_model = BERTopic(
    vectorizer_model=vectorizer_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model, 
    embedding_model=embedding_model,
    verbose=True,
    calculate_probabilities=True
)

embeddings = embedding_model.encode(texts, show_progress_bar=False)

topics, probs = topic_model.fit_transform(texts, embeddings)

2025-10-20 16:14:51,271 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-10-20 16:15:16,576 - BERTopic - Dimensionality - Completed ✓
2025-10-20 16:15:16,577 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-10-20 16:15:18,786 - BERTopic - Cluster - Completed ✓
2025-10-20 16:15:18,790 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-10-20 16:15:21,571 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,5461,-1_ახალი_წლის_უფრო_ყველა,"[ახალი, წლის, უფრო, ყველა, შესახებ, ადამიანის,...",[ბორის ჯონსონის თქმ უკრაინა შეჭრამდე პუტინი ბრ...
1,0,2494,0_საქართველოს_საქართველო_საერთაშორისო_ტექნოლოგი,"[საქართველოს, საქართველო, საერთაშორისო, ტექნოლ...",[ტაქს სამ მეტი ადამიანის გადაადგილ აკრძალულია ...
2,1,497,1_ხელოვნურ_ადამიანის_უფრო_ინტელექტს,"[ხელოვნურ, ადამიანის, უფრო, ინტელექტს, შეიძლ, ...",[რამდენად შორს ვართ ხელოვნური ზოგადი ინტელექტი...
3,2,469,2_მარკეტინგის_მონაცემთა_უფრო_მომხმარებელთა,"[მარკეტინგის, მონაცემთა, უფრო, მომხმარებელთა, ...",[ტექნოლოგი მარკეტინგ ინტეგრირ მთლიანად ცვლის მ...
4,3,354,3_სამუშაო_უფრო_რომლებიც_სამუშაო ადგილ,"[სამუშაო, უფრო, რომლებიც, სამუშაო ადგილ, დიდი,...",[გლობალურ სამუშაო ბაზარს მომდევნო ხუთი წლის გა...
...,...,...,...,...,...
60,59,45,59_ფილმის_ფანტასტიკა_ტომ_ფილმი,"[ფილმის, ფანტასტიკა, ტომ, ფილმი, მძაფრსიუჟეტია...",[ფილმ მიზნად დავისახეთ გვეჩვენებინა ქართული ყო...
61,60,43,60_უზრუნველყოფას_პროგრამულ უზრუნველყოფას_პროგრ...,"[უზრუნველყოფას, პროგრამულ უზრუნველყოფას, პროგრ...",[გამოიყენებს რაიმე პროგრამულ უზრუნველყოფას აღჭ...
62,61,43,61_შესწავლა_deep_მანქან_ინტელექტს,"[შესწავლა, deep, მანქან, ინტელექტს, კვანტური, ...",[deepmind განავითარა ფეხბურთის თამაშს სწავლობს...
63,62,43,62_კლიმატური ცვლილებ_ცხოველი_იშვიათი ფოტო_აუცი...,"[კლიმატური ცვლილებ, ცხოველი, იშვიათი ფოტო, აუც...",[კატ სურათები საბჭოთა საახალწლო ფოტოებ ჩააფოტო...


In [ ]:
# First pass: Use c-TF-IDF strategy with a threshold
new_topics = topic_model.reduce_outliers(texts, topics, strategy="c-tf-idf", threshold=0.15)

# Second pass: Use the embeddings strategy
new_topics = topic_model.reduce_outliers(texts, new_topics, strategy="embeddings", embeddings=embeddings)

# Final pass: Use the probabilities strategy for any remaining outliers
new_topics = topic_model.reduce_outliers(texts, new_topics, strategy="probabilities", probabilities=probs)
topic_model.update_topics(texts, topics=new_topics)
topic_model.get_topic_info()

2025-10-20 16:15:22,991 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


,Topic,Count,Name,Representation,Representative_Docs
0,0,2782,0_საქართველოს_საქართველო_საერთაშორისო_ტექნოლოგი,"[საქართველოს, საქართველო, საერთაშორისო, ტექნოლ...",[ტაქს სამ მეტი ადამიანის გადაადგილ აკრძალულია ...
1,1,951,1_ადამიანის_ხელოვნურ_უფრო_მაგრამ,"[ადამიანის, ხელოვნურ, უფრო, მაგრამ, შეიძლ, თუმ...",[რამდენად შორს ვართ ხელოვნური ზოგადი ინტელექტი...
2,2,765,2_უფრო_მარკეტინგის_მომხმარებლის_მონაცემთა,"[უფრო, მარკეტინგის, მომხმარებლის, მონაცემთა, მ...",[ტექნოლოგი მარკეტინგ ინტეგრირ მთლიანად ცვლის მ...
3,3,673,3_სამუშაო_უფრო_რომლებიც_დიდი,"[სამუშაო, უფრო, რომლებიც, დიდი, განვითარ, სფერ...",[გლობალურ სამუშაო ბაზარს მომდევნო ხუთი წლის გა...
4,4,366,4_chatgpt_chatgptის_openai_chatgptს,"[chatgpt, chatgptის, openai, chatgptს, openaiი...",[openai უნივერსალური ინტელექტის ჰუმანოიდ რობოტ...
...,...,...,...,...,...
59,59,161,59_ფილმის_ტომ_ფანტასტიკა_ფილმი,"[ფილმის, ტომ, ფანტასტიკა, ფილმი, დრა, ფილმ, მო...",[ფილმ მიზნად დავისახეთ გვეჩვენებინა ქართული ყო...
60,60,113,60_უზრუნველყოფას_პროგრამულ_მიმართება_აღჭურვილია,"[უზრუნველყოფას, პროგრამულ, მიმართება, აღჭურვილ...",[გამოიყენებს რაიმე პროგრამულ უზრუნველყოფას აღჭ...
61,61,107,61_წაკითხვა_შესწავლა_deep_მანქან,"[წაკითხვა, შესწავლა, deep, მანქან, გრძნობები, ...",[deepmind განავითარა ფეხბურთის თამაშს სწავლობს...
62,62,80,62_2018_ფოტო_მოგწამლათ_გაარკვიოთ,"[2018, ფოტო, მოგწამლათ, გაარკვიოთ, არქიტექტურუ...",[კატ სურათები საბჭოთა საახალწლო ფოტოებ ჩააფოტო...


In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_topics()

In [ ]:
# Filter the dataframe but keep track of original indices
df_time = df[df['fetch_time'].notnull()].copy()
df_time = df_time[df_time['processed_text'].notnull() & (df_time['processed_text'] != '')].copy()

# Store the original indices before resetting
original_indices = df_time.index.tolist()

# Now reset the index
df_time = df_time.reset_index(drop=True)

# Extract data for time plotting
texts_time = df_time['processed_text'].tolist()
timestamps = pd.to_datetime(df_time['fetch_time'], format='mixed', utc=True).dt.tz_convert(None).tolist()

# Match topics to the filtered documents using original indices
filtered_topics = [new_topics[i] for i in original_indices]

# Verify lengths match (optional debugging)
print(f"Length of texts_time: {len(texts_time)}")
print(f"Length of timestamps: {len(timestamps)}")
print(f"Length of filtered_topics: {len(filtered_topics)}")

# Run the temporal analysis
topics_over_time = topic_model.topics_over_time(nr_bins=20, global_tuning=True, evolution_tuning=True, docs=texts_time, timestamps=timestamps, topics=filtered_topics)

Length of texts_time: 8113
Length of timestamps: 8113
Length of filtered_topics: 8113


16it [00:03,  4.10it/s]


In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, topics=[9], custom_labels=True)